<a href="https://colab.research.google.com/github/poyu105/predictiveModel-senior_project-/blob/main/%E6%A2%81%E5%81%89%E8%88%AA/%E6%B3%A1%E9%BA%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# 加載數據（確保你已經讀取並清理好數據）
df_cleaned = pd.read_excel("畢業專題-預測備料資料2.xlsx", usecols=["季節", "天氣", "口味", "性別", "身高", "體重", "穿著", "湯麵或乾麵"])

# 確保所有相關列都是類別型態
for col in ["季節", "天氣", "口味", "性別", "穿著", "湯麵或乾麵"]:
    df_cleaned[col] = df_cleaned[col].astype('category')


# One-Hot 編碼特徵（季節 & 天氣）
X_encoded = pd.get_dummies(df_cleaned[["季節", "天氣", "口味", "性別", "身高", "體重", "穿著", "湯麵或乾麵"]])

# One-Hot 編碼標籤（泡麵種類）
# Assuming y_encoded_flavor and y_encoded_type are your new label columns
y_encoded_flavor = pd.get_dummies(df_cleaned["口味"])  # Flavor labels
y_encoded_type = pd.get_dummies(df_cleaned["湯麵或乾麵"]) # Type labels

# 將所有數值欄位轉換為數值類型
X_encoded = X_encoded.astype(float)
y_encoded_flavor = y_encoded_flavor.astype(float)
y_encoded_type = y_encoded_type.astype(float)

# 顯示轉換後的數據
print(X_encoded.head())
print(y_encoded_flavor.head())
print(y_encoded_type.head())

      身高     體重  季節_冬  季節_夏  季節_春  季節_秋  天氣_晴天  天氣_陰天  天氣_雨天  口味_海鮮  ...  \
0  178.0  100.0   0.0   0.0   1.0   0.0    0.0    0.0    1.0    1.0  ...   
1  161.0   50.0   0.0   0.0   1.0   0.0    0.0    1.0    0.0    0.0  ...   
2  161.0   50.0   0.0   0.0   1.0   0.0    0.0    0.0    1.0    0.0  ...   
3  154.0   41.0   0.0   0.0   1.0   0.0    1.0    0.0    0.0    1.0  ...   
4  154.0   41.0   0.0   0.0   1.0   0.0    0.0    0.0    1.0    0.0  ...   

   口味_牛肉  口味_素食  口味_辣味  性別_女  性別_男  穿著_休閒  穿著_正裝  穿著_運動  湯麵或乾麵_乾麵  湯麵或乾麵_湯麵  
0    0.0    0.0    0.0   0.0   1.0    0.0    0.0    1.0       0.0       1.0  
1    0.0    0.0    1.0   1.0   0.0    0.0    0.0    1.0       1.0       0.0  
2    0.0    0.0    0.0   1.0   0.0    1.0    0.0    0.0       1.0       0.0  
3    0.0    0.0    0.0   1.0   0.0    0.0    1.0    0.0       0.0       1.0  
4    0.0    1.0    0.0   1.0   0.0    1.0    0.0    0.0       0.0       1.0  

[5 rows x 21 columns]
    海鮮   清湯   牛肉   素食   辣味
0  1.0  0.0  0.0  0.0  0.

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# 定義模型
# Create two output layers for flavor and type
# Define inputs explicitly
input_layer = keras.Input(shape=(X_encoded.shape[1],))

# Shared hidden layers
hidden1 = layers.Dense(64, activation='relu')(input_layer)
hidden2 = layers.Dense(32, activation='relu')(hidden1)

# Output for flavor
flavor_output = layers.Dense(y_encoded_flavor.shape[1], activation='softmax', name='flavor_output')(hidden2)
# Output for type
type_output = layers.Dense(y_encoded_type.shape[1], activation='softmax', name='type_output')(hidden2)

# Create the model with multiple outputs
model = keras.Model(inputs=input_layer, outputs=[flavor_output, type_output])

# 編譯模型
# Compile with two outputs and losses
model.compile(optimizer='adam',
              loss={'flavor_output': 'categorical_crossentropy', 'type_output': 'categorical_crossentropy'},
              metrics={'flavor_output': 'accuracy', 'type_output': 'accuracy'})

# 訓練模型
# Fit with two target labels
model.fit(X_encoded.values,
          {'flavor_output': y_encoded_flavor.values, 'type_output': y_encoded_type.values},
          epochs=50, batch_size=16)

# 儲存模型（可選）
model.save("noodle_recommender_multi_output.h5")

Epoch 1/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - flavor_output_accuracy: 0.1698 - flavor_output_loss: 12.6142 - loss: 13.4182 - type_output_accuracy: 0.5737 - type_output_loss: 0.7756
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - flavor_output_accuracy: 0.3005 - flavor_output_loss: 2.7362 - loss: 3.4704 - type_output_accuracy: 0.5498 - type_output_loss: 0.7343 
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - flavor_output_accuracy: 0.1889 - flavor_output_loss: 2.1599 - loss: 2.9170 - type_output_accuracy: 0.5539 - type_output_loss: 0.7554 
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - flavor_output_accuracy: 0.2899 - flavor_output_loss: 1.9099 - loss: 2.5818 - type_output_accuracy: 0.5800 - type_output_loss: 0.6721 
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - flavor_output_accuracy: 0.3565 - flavor_output_loss: 1.5805 - loss: 2.2231 - type_output_accuracy: 0.6181 - type_output_loss: 0.6414 
Epoch 6/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - flavor_output_acc

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score
from tensorflow import keras
import pandas as pd # Import pandas for reading Excel file

# Load the saved model
model = keras.models.load_model("noodle_recommender_multi_output.h5")

# Import and preprocess validation data
df_validation = pd.read_excel("validation_data.xlsx", usecols=["季節", "天氣", "口味", "性別", "身高", "體重", "穿著", "湯麵或乾麵"])
for col in ["季節", "天氣", "口味", "性別", "穿著", "湯麵或乾麵"]:
    df_validation[col] = df_validation[col].astype('category')
X_val = pd.get_dummies(df_validation[["季節", "天氣", "口味", "性別", "身高", "體重", "穿著", "湯麵或乾麵"]]).astype(float)
y_val_flavor = pd.get_dummies(df_validation["口味"]).astype(float) # Flavor labels
y_val_type = pd.get_dummies(df_validation["湯麵或乾麵"]).astype(float) # Type labels

# Make predictions
predictions = model.predict(X_val)

# Predictions for flavor and type
y_pred_flavor = predictions[0]  # Assuming flavor is the first output
y_pred_type = predictions[1]  # Assuming type is the second output

# Get predicted and true class labels
y_pred_flavor_classes = np.argmax(y_pred_flavor, axis=1)
y_true_flavor_classes = np.argmax(y_val_flavor, axis=1)
y_pred_type_classes = np.argmax(y_pred_type, axis=1)
y_true_type_classes = np.argmax(y_val_type, axis=1)

# Calculate accuracy for flavor and type
accuracy_flavor = accuracy_score(y_true_flavor_classes, y_pred_flavor_classes)
accuracy_type = accuracy_score(y_true_type_classes, y_pred_type_classes)

print("Flavor Prediction Accuracy:", accuracy_flavor)
print("Type Prediction Accuracy:", accuracy_type)